In [5]:
import torch
import torch.nn as nn
import numpy as np
import os
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import torchvision

In [6]:

class ResBlock(nn.Module):
    def __init__(self, n_features):
        super(ResBlock,self).__init__()


        self.layers = nn.Sequential(
            nn.Conv2d(n_features, n_features, 3, 1, 1, bias=False),
            nn.BatchNorm2d(n_features),
            nn.PReLU(n_features),
            nn.Conv2d(n_features, n_features, 3, 1, 1, bias=False),
            nn.BatchNorm2d(n_features),
            nn.PReLU(n_features),
        )

    def forward(self,X):
        return self.layers(X) + X

class Body(nn.Module):
    def __init__(self, n_features, n_blocks):
        super(Body, self).__init__()

        self.n_features = n_features
        self.n_blocks = n_blocks

        self.blocks = nn.Sequential(*[ResBlock(n_features) for _ in range(n_blocks)])

    def forward(self, X):
        return self.blocks(X)



class Generator(nn.Module):
    
    def __init__(self, device, n_channels, n_features, n_blocks):
        super(Generator, self).__init__()
        def upsample(n_features, n_features_up, up_factor=2):
            return nn.Sequential(
                nn.Conv2d(n_features, n_features_up, 3, 1, 1, bias=False),
                nn.PixelShuffle(up_factor),
                nn.PReLU(n_features_up//4)
        )
        self.device = device
        self.n_channels = n_channels
        self.n_features = n_features
        self.n_blocks = n_blocks

        self.in_conv = nn.Sequential(
            nn.Conv2d(n_channels, n_features, 9, 1, 4,bias=False),
            nn.PReLU(n_features)
        )

        self.body = Body(n_features=n_features, n_blocks=n_blocks)

        self.final_conv = nn.Sequential(
            nn.Conv2d(n_features, n_features, 3,1,1,bias=False),
            nn.BatchNorm2d(n_features)
        )

        self.up_sample = nn.Sequential(
            upsample(n_features, n_features*16),
            upsample(n_features*4, n_features*16)
        )
        self.out = nn.Sequential(
            nn.Conv2d(n_features*4, n_channels, 9,1,4,bias=False),
        )


    
    def forward(self, X):
        X = self.in_conv(X)
        X = self.final_conv(self.body(X)) + X
        X = self.up_sample(X)
        return self.out(X)


class DiscBlock(nn.Module):
    def __init__(self, n_features_in, n_features_out, stride, padding):
        super(DiscBlock, self).__init__()
        
        self.net = nn.Sequential(
            nn.Conv2d(n_features_in, n_features_out, 3, stride=stride,padding=padding,bias=False),
            nn.BatchNorm2d(n_features_out),
            nn.LeakyReLU(0.2)
        )

    def forward(self, X):
        return self.net(X)

class Discriminator(nn.Module):
    def __init__(self, n_channels, n_features):
        super(Discriminator, self).__init__()
        
        self.in_conv = nn.Sequential(
            nn.Conv2d(n_channels, n_features, 3, 1,1,bias=False),
            nn.LeakyReLU(0.2)
        )

        self.layers = nn.Sequential(
            DiscBlock(n_features, n_features*2, 3,1),
            DiscBlock(n_features*2, n_features*2, 3,1),
            DiscBlock(n_features*2, n_features*4, 3,1),
            DiscBlock(n_features*4, n_features*4, 3,1),
            DiscBlock(n_features*4, n_features*8, 3,1),
            DiscBlock(n_features*8, n_features*8, 3,1),
            nn.Flatten(1,-1),
            nn.Linear(n_features*8, n_features*16),
            nn.LeakyReLU(0.2),
            nn.Linear(n_features*16, 1),
            nn.Sigmoid(),
            nn.Flatten(0,-1)
        )

    def forward(self, X):
        X = self.in_conv(X)
        return self.layers(X)

    

In [7]:
import os
from turtle import down
from PIL import Image




class CustomSet(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)

        self.transform=transform

        self.items = [os.path.join(self.root_dir, each) for each in self.file_list if not each.startswith(".")]

    def loader(self, path):
        img = Image.open(path)
        img.convert("RGB")
        return img


    def __getitem__(self,idx):
        img = self.loader(self.items[idx])

        if self.transform:
            img = self.transform(img)

        downsampled_tf = torchvision.transforms.Compose([torchvision.transforms.Resize(32, interpolation=torchvision.transforms.InterpolationMode.BICUBIC), torchvision.transforms.Normalize((0,0,0), (1,1,1))])
        downsampled = downsampled_tf(img)

        img = torchvision.transforms.functional.normalize(img, 0.5,0.5)

        return downsampled, img

    def __len__(self):
        return len(self.items)


transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.RandomCrop((128, 128), pad_if_needed=True)])

dset = CustomSet("./data/mirflickr/img", transform=transforms)

loader = torch.utils.data.DataLoader(dset, batch_size=32, shuffle=True)

In [8]:
import dotenv

f = open(".env", "w")
f.write('PYTORCH_ENABLE_MPS_FALLBACK="1"')
f.close()
dotenv.load_dotenv("../.env", override=True)
device = torch.device("mps")


num_epochs= 10


gen = Generator(device, 3, 64, 16).to(device)

disc = Discriminator(3, 64).to(device)

optim_gen = torch.optim.Adam(gen.parameters(), lr=0.0003, betas=(0.9, 0.999))
gen_losses = []

for ep in range(num_epochs):
    print(f"Starting epoch {ep}")
    for src, tgt in tqdm(loader):
        src = src.to(device)
        tgt = tgt.to(device)

        # print(src.shape, tgt.shape)
        preds = gen(src)
        # print(preds.shape)

        recon_loss = torch.nn.functional.mse_loss(preds, tgt)
        # print(recon_loss.item())
        gen_losses.append(recon_loss.item())
        recon_loss.backward()
        optim_gen.step()
        

        out = disc(preds)
        # print(out)
        # break


Starting epoch 0


  0%|          | 0/782 [00:00<?, ?it/s]/Users/doriclink/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/activation.py:1226: UserWarning: The operator 'aten::prelu' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  return F.prelu(input, self.weight)
  1%|          | 9/782 [00:22<32:00,  2.48s/it]


KeyboardInterrupt: 

In [ ]:
model = torchvision.models.vgg16_bn()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [10]:



def countconstruct(target: str, bank:list[str], cache=None):
    if cache is None:
        cache = {}

    if target in cache:
        return cache[target]
    if target == "":
        return 1

    can = 0

    for each in bank:
        if target.startswith(each):
            el = countconstruct(target[len(each):], bank, cache)
            can+=el
    cache[target] = can
    return can


In [11]:
countconstruct("happyness", ["happy", "ness", "ha", "pp", "y", "yness"])

3